In [2]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, parent_dir)

from CurveDataFetcher import CurveDataFetcher 

from dotenv import dotenv_values
env_path = os.path.join(parent_dir, ".env")
config = dotenv_values(env_path)

from datetime import datetime

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

# Data Fetching with `CurveDataFetcher`:
- Timeseries by CUSIP
- CT EOD, Bid, Offer, Mid Yields sourced from FedInvest
- CMT Yields from FRED (Fed) or Treasury - Both sources likely pull from the same/use same par curve build (Treasury's monotone convex interpolation performed on forward rates from bids around 3:30 PM EST by FRBNY)
- Pull All Active CUSIPs given date

In [3]:
cusip_curve_builder = CurveDataFetcher(use_ust_issue_date=True, fred_api_key=config["FRED_API_KEY"])

# Fetching Timeseries data by CUSIPs: 
Sources:
- FedInvest
- FedInvest (Hosted on Github)
- Public.com
- Bondsupermart
- Cbond
- Todos:
  - Yahoo finance (via Dusseldorf, Munich, Stuttgart, TLO) (downside all in CEST)
  - directly from the euro exchanges (in CEST)
  - Webull
  - other brokers?

### Directly from FedInvest - use `CurveDataFetcher.fedinvest_data_fetcher.cusips_timeseries`:

In [5]:
cusip_dict_df = cusip_curve_builder.fedinvest_data_fetcher.cusips_timeseries(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 8, 20),
    cusips=["912810UA4", "912810UC0", "912810ES3"],
)
cusip_dict_df

{'912810UA4':          Date  bid_price  offer_price   mid_price  eod_price  bid_yield  \
 0  2024-05-15  101.68750   101.687500  101.687500  101.78125   4.521660   
 1  2024-05-16  102.00000   102.015625  102.007812  101.71875   4.502775   
 2  2024-05-17  101.31250   101.312500  101.312500  101.03125   4.544326   
 3  2024-05-20  100.68750   100.703125  100.695312  100.78125   4.582495   
 4  2024-05-21  101.31250   101.312500  101.312500  101.03125   4.544288   
 ..        ...        ...          ...         ...        ...        ...   
 62 2024-08-14  108.62500   108.640625  108.632812  108.56250   4.118753   
 63 2024-08-15  107.21875   107.234375  107.226562  107.50000   4.197169   
 64 2024-08-16  107.56250   107.578125  107.570312  107.90625   4.177803   
 65 2024-08-19  108.53125   108.562500  108.546875  108.62500   4.123828   
 66 2024-08-20  109.15625   109.171875  109.164062  109.37500   4.089355   
 
     offer_yield  eod_yield  
 0      4.521660   4.515992  
 1      4.501

### Directly from Public.com - use `CurveDataFetcher.publicdotcom_data_fetcher.public_dotcom_timeseries_api`:

In [7]:
cusip_dict_df = cusip_curve_builder.publicdotcom_data_fetcher.public_dotcom_timeseries_api(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 8, 20),
    cusips=["912810UA4", "912810UC0", "912810ES3"],
)
cusip_dict_df

{'912810UA4':                         Date     Price       YTM
 0  2024-05-16 13:30:00+00:00  101.6930  4.521000
 1  2024-05-17 13:30:00+00:00  101.0625  4.560000
 2  2024-05-20 13:30:00+00:00  100.7205  4.580500
 3  2024-05-21 13:30:00+00:00  101.2555  4.548000
 4  2024-05-22 13:30:00+00:00  101.3360  4.543000
 ..                       ...       ...       ...
 61 2024-08-14 13:30:00+00:00  108.4344  4.129614
 62 2024-08-15 13:30:00+00:00  107.3515  4.190031
 63 2024-08-16 13:30:00+00:00  107.9047  4.158988
 64 2024-08-19 13:30:00+00:00  108.2903  4.137499
 65 2024-08-20 13:30:00+00:00  109.2759  4.083097
 
 [66 rows x 3 columns],
 '912810UC0':                        Date     Price       YTM
 0 2024-08-16 13:30:00+00:00  101.6405  4.146500
 1 2024-08-19 13:30:00+00:00  102.0548  4.129847
 2 2024-08-20 13:30:00+00:00  103.1252  4.068601,
 '912810ES3':                           Date     Price       YTM
 1085 2024-01-02 14:30:00+00:00  102.5495  4.454500
 1086 2024-01-03 14:30:00+00:00  1

# Fetching CT Yields:
- CT Yields: Market observed yields of on-the-runs (most recent issue for respective bill, note, bonds) 
- Default yields are EOD price markings according to FedInvest converted to YTM with rateslibs
- Able to enable bid, offer, mide side yields as of ~3:30 PM CST
- All tenors are returned by default, able to pass in specific tenor strings

In [8]:
ct_yields_df = cusip_curve_builder.fedinvest_data_fetcher.get_historical_ct_yields(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 8, 19),
)
ct_yields_df

,Date,8-Week,17-Week,26-Week,52-Week,2-Year,3-Year,5-Year,7-Year,10-Year,20-Year,30-Year
0,2024-01-02,NaN,5.414579,5.266761,4.812030,4.332596,4.100865,3.930802,3.951168,3.943937,4.247731,4.083212
1,2024-01-03,NaN,5.403556,5.277076,4.822615,4.332576,4.077939,3.895921,3.914869,3.906323,4.209600,4.052949
2,2024-01-04,NaN,5.403156,5.250537,4.855018,4.382398,4.145585,3.972914,3.992895,3.988823,4.295010,4.137418
3,2024-01-05,NaN,5.391337,5.238747,4.854037,4.399235,4.178916,4.015292,4.040103,4.048999,4.363246,4.207898
4,2024-01-08,NaN,5.390940,5.227726,4.831904,4.365890,4.121773,3.966166,3.993101,4.010992,4.322075,4.176737
...,...,...,...,...,...,...,...,...,...,...,...,...
154,2024-08-13,NaN,5.170481,4.984385,4.407569,3.939084,3.768494,3.679846,3.734021,3.847008,4.243176,4.162170
155,2024-08-14,NaN,5.201923,5.005497,4.428822,3.955043,3.767915,3.672761,3.718660,3.828016,4.209080,4.122208
156,2024-08-15,NaN,5.212155,5.050511,4.536665,4.087890,3.894849,3.790106,3.825287,3.916861,4.277370,4.176518
157,2024-08-16,NaN,5.189838,5.027992,4.503012,4.069662,3.872618,3.768861,3.804479,3.890126,4.258974,4.152863


# Fetching CMT Yields:

In [13]:
cmt_yields_fred_df = cusip_curve_builder.fred_data_fetcher.get_historical_cmt_yields(
    start_date=datetime(2000, 1, 1),
    end_date=datetime(2024, 8, 19),
)
cmt_yields_fred_df

Fetching from FRED...


,Date,13-Week,26-Week,52-Week,2-Year,3-Year,5-Year,7-Year,10-Year,20-Year,30-Year
0,2000-01-03,5.27,5.55,6.09,6.38,6.42,6.50,6.65,6.58,6.94,6.61
1,2000-01-04,5.27,5.50,6.00,6.30,6.34,6.40,6.56,6.49,6.84,6.53
2,2000-01-05,5.28,5.49,6.05,6.38,6.43,6.51,6.68,6.62,6.95,6.64
3,2000-01-06,5.25,5.44,6.03,6.35,6.39,6.46,6.63,6.57,6.86,6.58
4,2000-01-07,5.22,5.42,6.00,6.31,6.35,6.42,6.58,6.52,6.82,6.55
...,...,...,...,...,...,...,...,...,...,...,...
6157,2024-08-13,5.06,4.75,4.40,3.93,3.75,3.68,3.74,3.85,4.25,4.16
6158,2024-08-14,5.07,4.77,4.42,3.94,3.76,3.67,3.72,3.83,4.22,4.12
6159,2024-08-15,5.09,4.82,4.52,4.08,3.90,3.79,3.83,3.92,4.28,4.18
6160,2024-08-16,5.08,4.80,4.49,4.06,3.87,3.77,3.81,3.89,4.26,4.15


## Advantage of using treaaury.gov source:
- Download xlsx
- Able to apply treasury long-term rates Extrapolation Factors: [UST's Daily Treasury Long-Term Rates](https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_long_term_rate&field_tdr_date_value_month=202408)

In [11]:
cmt_yields_ust_df = cusip_curve_builder.ust_data_fetcher.get_historical_cmt_yields(
    start_date=datetime(2000, 1, 1),
    end_date=datetime(2024, 8, 19),
    treasury_data_dir=r"C:\Users\chris\Curvy-CUSIPs\data\treasury", # change to path you want
    download_treasury_par_yields=True,
    apply_long_term_extrapolation_factor=True, # Only enable if start_date is older than February 9, 2006
)
cmt_yields_ust_df

Fetching from treasury.gov...


,Date,4-Week,8-Week,13-Week,17-Week,26-Week,52-Week,2-Year,3-Year,5-Year,7-Year,10-Year,20-Year,30-Year
0,2001-01-02,NaN,NaN,5.87,NaN,5.58,5.11,4.87,4.82,4.76,4.97,4.92,5.46,5.35
1,2001-01-03,NaN,NaN,5.69,NaN,5.44,5.04,4.92,4.92,4.94,5.18,5.14,5.62,5.49
2,2001-01-04,NaN,NaN,5.37,NaN,5.20,4.82,4.77,4.78,4.82,5.07,5.03,5.56,5.44
3,2001-01-05,NaN,NaN,5.12,NaN,4.98,4.60,4.56,4.57,4.66,4.93,4.93,5.50,5.41
4,2001-01-08,NaN,NaN,5.19,NaN,5.03,4.61,4.54,4.55,4.65,4.94,4.94,5.52,5.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5907,2024-08-13,5.48,5.39,5.32,5.18,4.97,4.40,3.93,3.75,3.68,3.74,3.85,4.25,4.16
5908,2024-08-14,5.49,5.39,5.32,5.21,5.00,4.42,3.94,3.76,3.67,3.72,3.83,4.22,4.12
5909,2024-08-15,5.53,5.40,5.34,5.22,5.04,4.52,4.08,3.90,3.79,3.83,3.92,4.28,4.18
5910,2024-08-16,5.53,5.40,5.33,5.21,5.02,4.49,4.06,3.87,3.77,3.81,3.89,4.26,4.15


# Pulling all Active CUSIPs given date (Bills, Notes, Bonds):
- Able to pull in respective CUSIP's auction results
- Latest (wrt paseed in as of date) STRIPping activity
- Latest (wrt pass in as of date) SOMA Holdings

In [14]:
curve_set_df = cusip_curve_builder.build_curve_set(
    as_of_date=datetime(2024, 8, 19),
    # include_auction_results=True,
    # include_soma_holdings=True,
    # include_stripping_activity=True,
    sorted=True, # recommended to enable this
    use_github=True, # recommended to enable this
)
curve_set_df

,cusip,security_type,auction_date,issue_date,maturity_date,time_to_maturity,int_rate,high_investment_rate,is_on_the_run,label,security_term,original_security_term,offer_price,offer_yield,bid_price,bid_yield,mid_price,mid_yield,eod_price,eod_yield
0,912797KY2,Bill,2024-07-18,2024-07-23,2024-08-20,0.002740,NaN,5.365000,False,"Aug 24s, 17-Week",4-Week,17-Week,0.000000,NaN,99.985500,NaN,49.992750,NaN,100.000000,NaN
1,912797KZ9,Bill,2024-07-25,2024-07-30,2024-08-27,0.021918,NaN,5.381000,False,"Aug 24s, 17-Week",4-Week,17-Week,99.883667,6.145361,99.883556,6.151317,99.883612,6.148339,99.898111,5.371803
2,912797LA3,Bill,2024-08-01,2024-08-06,2024-09-03,0.041096,NaN,5.381000,False,"Sep 24s, 17-Week",4-Week,17-Week,99.781875,5.822992,99.781667,5.828631,99.781771,5.825811,99.796222,5.434437
3,912797LG0,Bill,2024-08-08,2024-08-13,2024-09-10,0.060274,NaN,5.381000,True,"Sep 24s, 17-Week",4-Week,17-Week,99.679778,5.700234,99.679778,5.700234,99.679778,5.700234,99.693750,5.447776
4,912797LH8,Bill,2024-07-18,2024-07-23,2024-09-17,0.079452,NaN,5.377000,False,"Sep 24s, 17-Week",8-Week,17-Week,99.576278,5.659341,99.575472,5.670280,99.575875,5.664811,99.590889,5.461146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,912810TT5,Bond,2023-10-12,2023-10-16,2053-08-15,29.008219,4.125,NaN,False,"Aug 53s, 30-Year",29-Year 10-Month,30-Year,99.812500,4.136096,99.781250,4.137960,99.796875,4.137028,99.843750,4.134234
389,912810TV0,Bond,2024-01-11,2024-01-16,2053-11-15,29.260274,4.750,NaN,False,"Nov 53s, 30-Year",29-Year 10-Month,30-Year,110.671875,4.118146,110.656250,4.119006,110.664062,4.118576,110.750000,4.113849
390,912810TX6,Bond,2024-04-11,2024-04-15,2054-02-15,29.512329,4.250,NaN,False,"Feb 54s, 30-Year",29-Year 10-Month,30-Year,102.015625,4.131068,102.000000,4.131977,102.007812,4.131523,102.062500,4.128343
391,912810UA4,Bond,2024-07-11,2024-07-15,2054-05-15,29.756164,4.625,NaN,False,"May 54s, 30-Year",29-Year 10-Month,30-Year,108.562500,4.122099,108.531250,4.123828,108.546875,4.122963,108.625000,4.118643
